# **Salomon Uran Parra C.C. 1015068767**

## **Laboratorio 7 - Aprendizaje Estadistico**

In [ ]:
import pandas as pd
import scipy as sc
import numpy as np
import h5py
import matplotlib.pylab as plt

In [ ]:
#solo se ejecuta una vez para cargar los archivos, luego se comenta

!wget https://github.com/CienciaDatosUdea/002_EstudiantesAprendizajeEstadistico/raw/f170fa0980192f9fe2ee5f2d7bd7962b4d66d517/semestre2025-2/Laboratorios/dataset/train_catvnoncat.h5
!wget https://github.com/CienciaDatosUdea/002_EstudiantesAprendizajeEstadistico/raw/f170fa0980192f9fe2ee5f2d7bd7962b4d66d517/semestre2025-2/Laboratorios/dataset/test_catvnoncat.h5


In [ ]:
data_train= "train_catvnoncat.h5"
train_dataset = h5py.File(data_train, "r")

data_test= "test_catvnoncat.h5"
test_dataset = h5py.File(data_test, "r")

# Read the data
xtrain_classes, xtrain, train_label =\
train_dataset["list_classes"],train_dataset["train_set_x"],train_dataset["train_set_y"]

test_classes, xtest,test_label =\
test_dataset["list_classes"],test_dataset["test_set_x"],test_dataset["test_set_y"]


xtrain_= np.reshape(xtrain,( 64*64*3 , 209))/255
xtest_ = np.reshape(xtest, ( 64*64*3 , 50))/255

ytrain_ = np.reshape(train_label,(1,209))
ytest_ = np.reshape(test_label,(1,50))

print(xtrain_.shape)
print(xtest_.shape)
print(ytrain_.shape)
print(ytest_.shape)

## Topología de la red.

1. Construir un clase  que permita definir una red neuronal con la topología
deseada y la función de activación para cada capa, para ello deberá construir una funcion Topology con el número de capas de la red neuronal :

Topology = [n_x, n_h1, n_h2, n_h3, ...,n_y]

En este caso:
- $n^{[0]}=n_x$ seran los valores de entradas de la capa de entrada
- $n^{[1]}=n_{h1}$ Primera capa oculta de la red neuronal
- $n^{[2]}=n_{h2}$ Segunda capa oculta de la red neuronal

.

.

.


- $n^{[l]}=n_{hl}$ Segunda capa oculta de la red neuronal
.

.

.

- $n^{[L]}=n_{y}$ Segunda capa oculta de la red neuronal

donde

- $\mathrm{n_x}$: valores de entrada
- $\mathrm{n_{h1}}$: hidden layer 1
- $\mathrm{n_{h2}}$: hidden layer 2
- $\mathrm{n_y}$: last layer

- $n^{[L]}=n_{y}$ Segunda capa oculta de la red neuronal


También definir una lista con las funciones de activaciones para cada capa.


activation=[None, relu, relu, relu, ...,sigmoid]

  


a. Cada unas de las capas deberá tener los parámetros de inicialización de manera aleatoria:


La matriz de parametros para cada capa debera tener:


$\mathrm{dim(\vec{b}^{[l]})}=n^{[l]}$

$\mathrm{dim(\vec{\Theta}^{[l]})}=n^{[l]}\times n^{[l-1]}$

Lo anteriores parametros deberán estar en el constructor de la clase.


b. Construya un metodo llamado output cuya salida serán los valores de Z y A


$\mathrm{dim(\vec{\cal{A}}^{[l]})}=n^{[l-1]}\times m $

$\mathrm{dim(\vec{\cal{Z}}^{[l]})}=n^{[l]}\times m $.

2. Construir un generalizacion de la red, en el que entrada el valor inicial
y la red neuronal completa arroje la salida y la actualizacion de la red con los parametros deseados:

  ```
  A, nn = forward_pass(A0, nn_red)

 ```
3. Encontrar la funcion de coste.


$$-\frac{1}{m} \sum\limits_{i = 1}^{m} (y^{(i)}\log\left(a^{[L] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right)) \tag{7}$$


4. Construir un codigo que permita realizar el BackwardPropagation

In [ ]:
def act_funcion(x, activation):
    if activation=="sigmoid":
      x_arr = np.asarray(x)
      f = 1/(1+np.exp(-x_arr))
      fp = f*(1-f)
      return np.array([f, fp])
      #creacion de la funcion sigmoide

    elif activation=="relu":
      x_arr = np.asarray(x)
      f = np.maximum(0, x_arr)
      fp = (x_arr > 0).astype(float)
      return np.array([f, fp])
      #creacion de la funcion relu

    elif activation is None or activation=="linear":
      x_arr = np.asarray(x)
      return np.array([x_arr, np.ones_like(x_arr)])
      #activacion lineal, osea no aplicar ninguna funcion

class layer_nn():
  def __init__(self, n_pr, n_be, activation):
    self.theta = 2*np.random.random((n_pr, n_be)) - 1
    self.b = 2*np.random.random((n_pr, 1))-1
    self.activation = activation
    self.dtheta = None
    self.db = None
    #se crean las caracteristicas dtheta y db para guardar los gradientes y actualizar luego los parametros

  def output(self, X): #aqui X es un input de dimension nb x m, es el output del layer previo
    self.Z = self.theta @ X + self.b #sirve aun si theta @ x es npr x m, va a devolver un npr x m con b sumado a cada columna
    self.A = act_funcion(self.Z, self.activation)[0] #devuelve tambien un npr x m con la funcion de activacion aplicada a cada elemento
    self.Ap = act_funcion(self.Z, self.activation)[1] #devuelve la derivada de la funcion de activacion, npr x m

  def update(self, learning_rate):
    self.theta = self.theta - learning_rate * self.dtheta
    self.b = self.b - learning_rate * self.db
    #actualiza los parametros de la capa dado un learning_rate

def cost_func(Y, A): #se le ingresara el label del target y el output del forward, A debe ser un array, Y ingresa como 1 x m y A tambien 1 x m,
    m = Y.shape[1]
    Y = np.array(Y)
    A = np.array(A)

    epsilon = 1e-10
    A = np.clip(A, epsilon, 1 - epsilon)
    #en caso de que A contenga 0s o 1s, los reemplaza por epsilon o 1-epsilon, para evitar problemas con el logaritmo

    ji = - ( Y*np.log(A) + (1-Y)*np.log(1-A)) #funcion de coste
    cost = ji.sum()
    return cost/m

class model_nn(): #la clase se define con una topologia y una lista de funciones de activacion para cada capa
    def __init__(self, topology, activation):
        self.topology = topology
        self.layers = []
        self.activation = activation

    def Topology(self): #construye la arquitectura de la red, osea capas y numero de neuronas por capa, ademas de su funcion de activacion
        for i in range(len(self.topology)-1):
            self.layers.append(layer_nn(self.topology[i+1],self.topology[i],self.activation[i]))

    def forward(self, X): #para un dato de entrada n0 x m (un dato de entrenamiento), hace el forward en los layers guardados
        self.layers[0].output(X)
        for i in range(len(self.layers)-1):
            self.layers[i+1].output(self.layers[i].A)
        return self.layers[-1].A #retorna una salida 1 x m

    def backward(self, X, Y, alpha = 0.01):
      #nuevamente, aqui X es una entrada n0 x m (input de entrenamiento) y Y es un dato 1xm que es el label del target, haremos un backward

        self.A_L = self.forward(X) #debe ser una salida 1 x m, un forward para cada dato de entrenamiento con n0 features

        self.cost = cost_func(Y, self.A_L) #funcion de coste para el forward realizado

        m = Y.shape[1] #dimension o numero de datos de entrenamiento

        L = len(self.layers) #numero de capas en la red

        epsilon = 1e-10
        self.A_L = np.clip(self.A_L, epsilon, 1 - epsilon) #evita problemas de division por cero en el dA_L

        dA_L = -(np.divide(Y, self.A_L) - np.divide(1 - Y, 1 - self.A_L)) #dA_L como esta definido

        fp_L = self.layers[L-1].Ap #derivada de la funcion de activacion en la ultima capa

        dZ_L = np.multiply(dA_L,fp_L) #dZ de la ultima capa


        if L == 1: #en caso de que haya una unica capa, el output de la capa anterior es X
            self.layers[L-1].dtheta = dZ_L @ X.T / m
        else: #output de la capa anterior
            self.layers[L-1].dtheta = dZ_L @ self.layers[L-2].A.T / m
            #son los gradientes para los parametros theta de la ultima capa

        self.layers[L-1].db = np.sum(dZ_L, axis=1, keepdims = True) / m #gradiente para los parametros b de la ultima capa

        for l in range(L-2, -1, -1): #se itera desde la penultima capa hasta la primera, al reves, backpropagation


            dA = np.dot(self.layers[l+1].theta.T, dZ_L) #dA de la capa l empleando los thetas de la capa l+1 y el dZ de la misma capa

            fp = self.layers[l].Ap #derivada de la funcion de activacion en la capa l
            dZ_L = np.multiply(dA, fp) # dZ para la capa l-esima

            if l == 0: #para la primera capa, el input es X
                self.layers[l].dtheta = dZ_L @ X.T / m
            else: #para las otras capas, se toma el output de la capa l-1
                self.layers[l].dtheta = dZ_L @ self.layers[l-1].A.T / m
                #gradiente los thetas en la capa l-esima

            self.layers[l].db = np.sum(dZ_L, axis=1,keepdims = True) / m #gradiente de los b en la capa l

        for l in range(L):
            self.layers[l].update(alpha)
            #actualizacion de los parametros en todas las capas dado un learning rate alpha

Ahora, una vez construida la clase del modelo de red neuronal, podemos construir la topologia de una red para hacer un clasificador de gatos empleando el dataset definido mas arriba:

In [ ]:
n_x = xtrain_.shape[0]
n_y = ytrain_.shape[0]

np.random.seed(1) #para tener reproducibilidad

topology = [n_x,  20, 15, 10,  n_y] #se define la topologia con el numero de inputs, de neuronas por capa oculta y de outputs

activations = ['relu', 'relu', 'relu', 'sigmoid'] #se definen las funciones de activacion para cada capa, relu para las ocultas, sigmoide para el output

mdl = model_nn(topology, activations) #se crea el modelo

mdl.Topology() #se crea la topologia

alpha = 0.0045 #learning rate
num_epochs = 3000 #numero de epocas

Asi pues, ensayaremos entrenar una red neuronal con el dataset xtrain con 3 capas ocultas y una capa de salida. Las capas intermedias tienen 20, 15 y 10 neuronas respectivamente con una funcion de activacion ReLu. La ultima capa sale con una funcion de activacion sigmoide. El modelo se entrenara con un learning rate de $\alpha = 0.0045$ para todas las capas y se realizaran un total de 3000 epocas de entrenamiento.

In [ ]:
#para las siguientes celdas de codigo, se empleo ayuda de la IA Gemini para la sintaxis y orden de la escritura
#la documentacion y ciertas correciones de funcionamiento del codigo se hicieron a mano

def calculate_accuracy(predictions, true_labels): #se le entrega la prediccion y el label verdadero (target)
    #funcion que permite calcular la precision de una prediccion porcentualmente
    binary_predictions = (predictions > 0.5).astype(float)
    accuracy = np.mean(binary_predictions == true_labels) * 100
    return accuracy

costs = []
train_accuracies = []
test_accuracies = []

for i in range(num_epochs):
    mdl.backward(xtrain_, ytrain_, alpha) #para la epoca actual, hacemos el forward, el backward y actualizamos los parametros de la red
    costs.append(mdl.cost) #guardamos la funcion de coste de la epoca actual

    train_predictions = mdl.forward(xtrain_)
    train_acc = calculate_accuracy(train_predictions, ytrain_)
    train_accuracies.append(train_acc)
    #se calcula la precision de la prediccion en el set de entrenamiento

    test_predictions = mdl.forward(xtest_)
    test_acc = calculate_accuracy(test_predictions, ytest_)
    test_accuracies.append(test_acc)
    #se calcula la precision de la prediccion en el set de test


    if i % 100 == 0:
        print(f"Epoch {i}: Cost = {mdl.cost:.4f}, Train Accuracy = {train_acc:.2f}%, Test Accuracy = {test_acc:.2f}%")
        #se imprime la precision en el entrenamiento y test por cada epoca

print(f"Training complete. Final Cost: {mdl.cost:.4f}")
print(f"Final Training Accuracy: {train_accuracies[-1]:.2f}%")
print(f"Final Test Accuracy: {test_accuracies[-1]:.2f}%")

En general, dependiendo de la semilla que se use para inicializar los pesos de la red, vemos en estas 3000 primeras epocas, el modelo tiende a un overfitting sobre los datos de entrenamiento, mientras que tiene fuertes dificultades para generalizar sobre los datos de test. Es decir, el modelo se esta "aprendiendo de memoria" los datos de entrenamiento.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 4))

ax[0].plot(costs)
ax[0].set_xlabel("Epoch")
ax[0].set_ylabel("Cost")
ax[0].set_title("Learning Curve (Cost)")
ax[0].grid(True)

ax[1].plot(train_accuracies, label='Train Accuracy')
ax[1].plot(test_accuracies, label='Test Accuracy')
ax[1].set_xlabel("Epoch")
ax[1].set_ylabel("Accuracy (%)")
ax[1].set_title("Learning Curve (Accuracy)")
ax[1].legend()
ax[1].grid(True)

plt.tight_layout()
plt.show()

Aqui podemos observar la evolucion tanto de la funcion de coste como de las graficas de aprendizaje del modelo. En el dataset de test, se queda estancado y practicamente adivina el resultado. En el dataset de entrenamiento lo va aprendiendo poco a poco, claro sintoma de overfitting.

In [ ]:
plt.imshow(xtrain[14])

In [ ]:
image_original = xtrain_[:, 14]
image_reshaped = image_original.reshape(n_x, 1)

prediction = mdl.forward(image_reshaped)
print(f"Prediccion de la imagen: {prediction}")

binary_prediction = (prediction > 0.5).astype(int)
print(f"Categoria en la que entra la clasificacion: {binary_prediction}")


In [ ]:
plt.imshow(xtrain[15])

In [ ]:
image_original = xtrain_[:, 15]
image_reshaped = image_original.reshape(n_x, 1)

prediction = mdl.forward(image_reshaped)
print(f"Prediccion de la imagen: {prediction}")

binary_prediction = (prediction > 0.5).astype(int)
print(f"Categoria en la que entra la clasificacion: {binary_prediction}")


En estas ultimas celdas, vemos la prediccion que lanza sobre una imagen que claramente es un gato y sobre otra que claramente no lo es. Esto muestra que esta aprendiendo bien los datos de entrenamiento, pero luego al observar el conjunto de test, sabemos que lo que esta haciendo es un overfitting. Habria que explorar tecnicas o topologias de la red que le permitan una mayor libertad de los parametros para generalizar mejor el modelo y poder performar mejor en los test. Recordar que la categoria 1 es para los gatos y 0 es para los no gatos.